# Imports

In [0]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image

# Basic GAN (FF+ MNIST)

In [25]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5,), 
                                     std=(0.5,))])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Start training
total_step = len(data_loader)

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    
    # Reshape sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)

    fake_images_clone = fake_images.clone()
    print(fake_images_clone[0])
    print(denorm(fake_images_clone[0]))
    im = denorm(fake_images_clone[0]).cpu().detach().numpy()
    im = (im * 255).astype(np.uint8)
    im = im.reshape((28, 28, 1))
    print(im)
    print(im.shape)
    print(type(im))
    Image.fromarray(im)
    Image.fromarray((denorm(fake_images_clone[0].cpu().detach().numpy()) * 255).astype(np.uint8))


Epoch [0/200], Step [200/600], d_loss: 0.0440, g_loss: 4.4528, D(x): 0.99, D(G(z)): 0.04
Epoch [0/200], Step [400/600], d_loss: 0.1548, g_loss: 5.4707, D(x): 0.98, D(G(z)): 0.09
Epoch [0/200], Step [600/600], d_loss: 0.0379, g_loss: 5.0501, D(x): 0.99, D(G(z)): 0.02
tensor([[[-9.9538e-01, -9.4843e-01, -9.9749e-01, -9.8430e-01,  8.3717e-01,
          -9.0982e-01,  4.4531e-01, -8.3549e-01, -9.9552e-01, -9.7917e-01,
          -9.3189e-01, -9.8849e-01, -9.8724e-01, -9.9805e-01, -9.9520e-01,
          -9.9667e-01, -9.9813e-01, -9.8274e-01, -9.9600e-01, -9.9660e-01,
          -9.9940e-01, -9.5012e-01, -9.4758e-01, -9.9837e-01,  4.5125e-01,
           4.9651e-01, -9.9445e-01, -9.7448e-01],
         [-9.8808e-01, -7.9424e-01, -8.3829e-01,  8.7330e-01, -9.9814e-01,
          -9.9838e-01, -9.9971e-01, -9.6643e-01, -9.7315e-01, -9.9258e-01,
          -9.9937e-01, -9.9177e-01, -9.2248e-01, -5.5532e-01, -9.1487e-01,
          -9.8042e-01, -9.8383e-01, -9.9553e-01, -9.9043e-01,  9.0875e-01,
        

TypeError: ignored